Este desarrollo pretende buscar los precios de unos fondos de inversion dados, calcular el valor en euros en funcion de las participaciones.

Para ello se crea lista con productos. Se buscan los productos en Morningstar y se almacena valor, moneda y fecha


EL proceso de busqueda consiste en entrar en la pagina web, saltarse la cookies e ir buscando cada producto de la lista, inicalizando previamente el campo de la busqueda de la pagina web.

Se concatenan el df inicial con fondos y participaciones con el de moneda, valor y fecha. Se calcula la columna importe de particiaciones por valor en la moneda original.

Se entra en una URl para obtener el tipo de cambio del doalr a euros y se crea una columna en el df con el valor del tipo de cambio 1 para los fondos cuyo valor es en euros y con el tipo de cambio obtebido para el dolar en aquellos que estan dolares.


Al final produce un fichero csv todos los datos.


In [1]:
#!pip install selenium

In [2]:
#!pip install fake-useragent

In [3]:
# !pip install datetime

In [4]:
# !pip install webdriver-manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [5]:
# !pip install pymupdf

In [6]:
import requests as req
import pandas as pd

import time
import datetime  # manejo temporal

#from bs4 import BeautifulSoup as bs

from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent

PATH = r'C:\Users\ragod\Desktop\reposicion\Proyectos_TB\Proyectos_TB\Funciones\chromedriver.exe'  #'driver/chromedriver'
usuario=UserAgent().random


from selenium import webdriver  # driver selenium
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By   # para elegir por elemento
from selenium.webdriver.support.ui import WebDriverWait # para el time sleep
from selenium.webdriver.support import expected_conditions as EC # condiciones esperadas
from selenium.webdriver import ActionChains as AC # acciones en cadena


In [7]:
url='https://www.morningstar.es/es/'
driver=webdriver.Chrome(PATH) 
driver.get(url)

time.sleep(3)


C:\Users\ragod\AppData\Local\Temp/ipykernel_7932/941672663.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)


In [8]:

#Lectura de fondos

mis_fondos =pd.read_excel(r'C:\Users\ragod\Desktop\reposicion\Proyectos_TB\Proyectos_TB\fondos\fondos.xlsx', sheet_name ='Fondos',header = 0)
mis_fondos_l =list (mis_fondos.n_nom_fondo)


In [9]:

#Busqueda en MorningStar
saltar_coockies=driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]')
saltar_coockies.click()
time.sleep(3)

aceptar_cookies=driver.find_element(By.XPATH,'//*[@id="btn_individual"]') # objeto en la web de busqueda
aceptar_cookies.click() # Boton busqueda


mi_lista_valorada =[]

#buscar fondos

for n in mis_fondos_l:
    buscar=driver.find_element(By.XPATH,'//*[@id="quoteSearch"]').clear()
    producto = n
    buscar=driver.find_element(By.XPATH,'//*[@id="quoteSearch"]')# objeto en la web de busqueda
    buscar.send_keys(producto)  # incrusta la string
    time.sleep(3)
    buscar=driver.find_element(By.XPATH,'//*[@id="GoSearch"]')
    buscar.click()
    buscar=driver.find_element(By.XPATH,'//*[@id="ctl00_MainContent_fundTable"]/tbody/tr[2]/td[1]/a')
    buscar.click()

    precio= (driver.find_element(By.XPATH, '//*[@id="overviewQuickstatsDiv"]/table/tbody/tr[2]/td[3]').text).split()
    fecha =driver.find_element(By.XPATH, '//*[@id="overviewQuickstatsDiv"]/table/tbody/tr[2]/td[1]').text
    fecha=fecha[3:]
    # Crear lista valorada 
    mi_lista_valorada.append([precio[0],precio[1],fecha])

                             




In [10]:
df = pd.DataFrame(mi_lista_valorada, columns = ['moneda','valor','fecha'])
df_final = pd.concat([mis_fondos,df],axis=1)
df_final['valor'] =df_final['valor'].replace(",",".", regex = True).astype(float)
df_final['importe']=df_final.valor*df_final.num_participaciones


In [23]:
url='https://www.xe.com/es/currencyconverter/convert/?Amount=1&From=USD&To=EUR'
driver=webdriver.Chrome(PATH) 
driver.get(url)

C:\Users\ragod\AppData\Local\Temp/ipykernel_7932/275695470.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)


In [24]:
cambio =(driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[2]/section/div[2]/div/main/form/div[2]/div[1]/p[2]').text)


In [25]:
t_cambio= cambio.split()[0].replace(',','.')


In [39]:
t_cambio

'0.94422247'

In [40]:
df_final['t_cambio']=df_final.apply(lambda x: 1 if x.moneda =='EUR' else t_cambio,axis=1).astype(float) 

In [41]:
df_final['importe_euro']=df_final.importe*df_final.t_cambio

In [43]:
suma_fondos=df_final.importe_euro.sum()

In [31]:
df_final

,n_nom_fondo,num_participaciones,moneda,valor,fecha,importe,t_cambio,importe_euro
0,BGF Euro Short Duration Bond A2 EUR,22.45,EUR,15.16,18/05/2022,340.3420,1.000000,340.342000
1,iShares Euro Govt Bd Idx (LU) A2 EUR,8.34,EUR,117.70,18/05/2022,981.6180,1.000000,981.618000
2,iShares Em Markets Govt Bd Idx (LU) A2 EURH,13.22,EUR,93.87,18/05/2022,1240.9614,1.000000,1240.961400
3,iShares Europe Equity Index (LU) A2 EUR,11.77,EUR,199.20,18/05/2022,2344.5840,1.000000,2344.584000
4,iShares Developed Real Estate Index Fund (IE) ...,31.32,EUR,18.57,18/05/2022,581.6124,1.000000,581.612400
5,iShares North America Eq Index (LU) A2 USD,7.41,USD,292.60,18/05/2022,2168.1660,0.944222,2047.231056
6,iShares Japan Equity Index (LU) A2 USD,6.22,USD,171.55,18/05/2022,1067.0410,0.944222,1007.524089
7,iShares Emerging Markets Eq Idx (LU) A2 USD,9.77,USD,125.80,18/05/2022,1229.0660,0.944222,1160.511734


In [62]:
df_final2=df_final[['n_nom_fondo', 'importe_euro']]

In [63]:
df_final2

,n_nom_fondo,importe_euro
0,BGF Euro Short Duration Bond A2 EUR,340.342000
1,iShares Euro Govt Bd Idx (LU) A2 EUR,981.618000
2,iShares Em Markets Govt Bd Idx (LU) A2 EURH,1240.961400
3,iShares Europe Equity Index (LU) A2 EUR,2344.584000
4,iShares Developed Real Estate Index Fund (IE) ...,581.612400
5,iShares North America Eq Index (LU) A2 USD,2047.231056
6,iShares Japan Equity Index (LU) A2 USD,1007.524089
7,iShares Emerging Markets Eq Idx (LU) A2 USD,1160.511734


In [64]:
total_fondos= df_final2.importe_euro.sum()

In [65]:
df3=df_final2.append({'n_nom_fondo':'total','importe_euro': total_fondos}, ignore_index = True)

In [67]:
df3

,n_nom_fondo,importe_euro
0,BGF Euro Short Duration Bond A2 EUR,340.342000
1,iShares Euro Govt Bd Idx (LU) A2 EUR,981.618000
2,iShares Em Markets Govt Bd Idx (LU) A2 EURH,1240.961400
3,iShares Europe Equity Index (LU) A2 EUR,2344.584000
4,iShares Developed Real Estate Index Fund (IE) ...,581.612400
5,iShares North America Eq Index (LU) A2 USD,2047.231056
6,iShares Japan Equity Index (LU) A2 USD,1007.524089
7,iShares Emerging Markets Eq Idx (LU) A2 USD,1160.511734
8,total,9704.384679


In [68]:
fecha = str(datetime.datetime.now())
fecha2 =fecha.split()[0].replace('-','')
archivo_con_fecha = f"{fecha2}_fondos.csv"
# Crea CSV
df3.to_csv(archivo_con_fecha, index = False)

In [66]:
print(f"El valor acumulado de los fondos es : {total_fondos:.2f}")

El valor acumulado de los fondos es : 9704.38


In [36]:

import smtplib
   
from email.MIMEText import MIMEText 
# Nos conectamos al servidor SMTP de Gmail 
serverSMTP = smtplib.SMTP('smtp.gmail.com',587) 
serverSMTP.ehlo() 
serverSMTP.starttls() 
serverSMTP.ehlo() 
serverSMTP.login(emisor,"xxx")      

    
    
emisor = "ragodmar@gmail.com" 
receptor = "ragodmar@gmail.com" 
   
# Configuracion del mail
texto_mensaje =f"El valor acumulado de los fondos es : {total_fondos:.2f}"
mensaje = MIMEText("texto_mensaje") 
mensaje['From']=emisor 
mensaje['To']=receptor 
mensaje['Subject']="Valor de los fondos" 

   
# Enviamos el mensaje 
serverSMTP.sendmail(emisor,receptor,mensaje.as_string()) 
 
# Cerramos la conexion 
serverSMTP.close()
'''

'\nimport smtplib\n   \nfrom email.MIMEText import MIMEText \n     \nemisor = "copia el mail de origen" \nreceptor = "copia el mail de destino" \n   \n# Configuracion del mail \nmensaje = MIMEText("Este correo ha sido enviado desde Python") \nmensaje[\'From\']=emisor \nmensaje[\'To\']=receptor \nmensaje[\'Subject\']="Valor de los fondos" \n# Nos conectamos al servidor SMTP de Gmail \nserverSMTP = smtplib.SMTP(\'smtp.gmail.com\',587) \nserverSMTP.ehlo() \nserverSMTP.starttls() \nserverSMTP.ehlo() \nserverSMTP.login(emisor,"ingresa la contraseña de tu correo con el que deseas enviar un mail") \n   \n# Enviamos el mensaje \nserverSMTP.sendmail(emisor,receptor,mensaje.as_string()) \n \n# Cerramos la conexion \nserverSMTP.close()\n'

In [69]:
import sqlite3

conexion=sqlite3.connect("bd1.db")
try:
    conexion.execute("""create table fondos (
                              codigo integer primary key autoincrement,
                              descripcion text,
                              precio real
                        )""")
    print("se creo la tabla fondos")                        
except sqlite3.OperationalError:
    print("La tabla articulos ya existe")                    
conexion.close()

se creo la tabla fondos


In [73]:
df3.to_sql('fondos',conexion,index=False)


ProgrammingError: Cannot operate on a closed database.